In [ ]:
# --- Cell: Measure Angle from Image ---
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.backend_bases import MouseButton

# 1. Load the image
# Make sure 'snouty.png' is in the same directory, or provide full path
try:
    img_path = "snouty.png"
    img = plt.imread(img_path)
except FileNotFoundError:
    print("❌ ERROR: Please ensure 'image.png' is in the current directory.")
    img = np.zeros((100, 100))  # Fallback so cell doesn't crash

# 2. Setup Plot
fig, ax = plt.subplots(figsize=(9, 7))
ax.imshow(img)
ax.set_title("Click 2 points for VERTICAL axis, then 2 points for ANGLED axis")
ax.axis("off")

# Global state
coords = []
lines = []


def calculate_angle(p1, p2, p3, p4):
    """Calculates the angle between two lines defined by points."""
    # Vector 1
    v1 = np.array([p2[0] - p1[0], p2[1] - p1[1]])
    # Vector 2
    v2 = np.array([p4[0] - p3[0], p4[1] - p3[1]])

    # Normalize vectors
    v1_u = v1 / np.linalg.norm(v1)
    v2_u = v2 / np.linalg.norm(v2)

    # Dot product
    dot_product = np.dot(v1_u, v2_u)

    # Clip to handle floating point errors slightly outside [-1, 1]
    dot_product = np.clip(dot_product, -1.0, 1.0)

    # Angle in radians
    angle_rad = np.arccos(dot_product)

    # Convert to degrees
    angle_deg = np.degrees(angle_rad)

    # We want the acute angle (between 0 and 90)
    if angle_deg > 90:
        angle_deg = 180 - angle_deg

    return angle_deg


def onclick(event):
    # Only accept left clicks within the axes
    if event.button is not MouseButton.LEFT or event.inaxes != ax:
        return

    coords.append((event.xdata, event.ydata))

    # Draw a small red dot where clicked
    ax.plot(event.xdata, event.ydata, "ro", markersize=4)

    n_points = len(coords)

    # --- Logic for drawing lines ---
    if n_points == 1:
        ax.set_title("Point 1 set. Click bottom of PRIMARY (Vertical) objective.")

    elif n_points == 2:
        # Draw first line (Primary Axis)
        p1, p2 = coords[0], coords[1]
        (line,) = ax.plot(
            [p1[0], p2[0]],
            [p1[1], p2[1]],
            "b-",
            linewidth=2,
            label="Reference (Vertical)",
        )
        lines.append(line)
        ax.set_title("Vertical axis set. Now click top of REMOTE (Angled) objective.")

    elif n_points == 3:
        ax.set_title("Point 3 set. Click bottom of REMOTE (Angled) objective.")

    elif n_points == 4:
        # Draw second line (Remote Axis)
        p3, p4 = coords[2], coords[3]
        (line,) = ax.plot(
            [p3[0], p4[0]], [p3[1], p4[1]], "g-", linewidth=2, label="Remote Axis"
        )
        lines.append(line)

        # Calculate
        angle = calculate_angle(coords[0], coords[1], coords[2], coords[3])

        # Display result
        ax.set_title(f"Measured Angle: {angle:.2f}°")
        ax.legend()
        print("\n--- MEASUREMENT RESULT ---")
        print(f"📐 Calculated Angle: {angle:.4f}°")

        # Cleanup to prevent further clicks messing it up
        fig.canvas.mpl_disconnect(cid)

    fig.canvas.draw()


# Connect the click event
cid = fig.canvas.mpl_connect("button_press_event", onclick)
plt.show()